In [1]:
import numpy as np
from scipy import signal
import numba
import librosa
import librosa.display
%matplotlib nbagg
import mir_eval.display
import mir_eval.sonify
from IPython.display import Audio
import matplotlib.pyplot as plt
import jams

In [2]:
@numba.jit(nopython=True)
def _viterbi(logp, logA, V, ptr, S):
    T, m = logp.shape
    
    V[0] = logp[0]
    
    for t in range(1, T):
        # Want V[t, j] <- p[t, j] * max_k V[t-1, k] * A[k, j]
        #    assume at time t-1 we were in state k
        #    transition k -> j

        # Broadcast over rows:
        #    Tout[k, j] = V[t-1, k] * A[k, j]
        #    then take the max over columns
        # 
        # We'll do this in log-space for stability

        Tout = V[t-1] + logA.T

        # Unroll the max/argmax loop to enable numba support
        for j in range(m):
            ptr[t, j] = np.argmax(Tout[j])
            V[t, j] = logp[t, j] + np.max(Tout[j])
            
    # Now roll backward
    
    # Get the last state
    S[-1] = np.argmax(V[-1])
    
    for t in range(T-2, -1, -1):
        S[t] = ptr[t+1, S[t+1]]
        
    return S, V, ptr


def viterbi(p, A):
    '''Viterbi decoding for discriminative HMMs
    
    Parameters
    ----------
    p : np.ndarray [shape=(T, m)], non-negative
        p[t] is the distribution over states at time t.
        Each row must sum to 1.
        
    A : np.ndarray [shape=(m, m)], non-negative
        A[i,j] is the probability of a transition from i->j.
        Each row must sum to 1.
        
    Returns
    -------
    s : np.ndarray [shape=(T,)]
        The most likely state sequence
    '''
    
    T, m = p.shape
    
    assert A.shape == (m, m)
    
    assert np.all(A >= 0)
    assert np.all(p >= 0)
    assert np.allclose(A.sum(axis=1), 1)
    assert np.allclose(p.sum(axis=1), 1)
    
    V = np.zeros((T, m), dtype=float)
    
    ptr = np.zeros((T, m), dtype=int)
    
    logA = np.log(A + 1e-10)
    logp = np.log(p + 1e-10)
    S = np.zeros(T, dtype=int)
    
    return _viterbi(logp, logA, V, ptr, S)

def make_self_transition(N, alpha=0.05):
    
    T = np.empty((N, N), dtype=float)
    T[:] = alpha / (N - 1)
    
    np.fill_diagonal(T, 1-alpha)
    
    return T

import scipy.signal

def make_bump_transition(N, window='hann', width=5, alpha=0.05):
    
    T = np.eye(N, dtype=float)
    w = librosa.filters.get_window(window, width, fftbins=False)[np.newaxis]
    T = scipy.signal.convolve(T, w, mode='same',)
    
    T = (1 - alpha) * T + alpha
    return T / T.sum(axis=1, keepdims=True)

In [4]:
def acf_to_voicing_prob(A, quantile=75, epsilon=5e-3):
    Q = np.percentile(A[0], quantile)
    V = np.minimum(A[0], Q) / (Q + epsilon)
    return V

def constrained_pitch_prob(y, string_number):
    str_midi_dict = {
        0: 40,
        1: 45,
        2: 50,
        3: 55,
        4: 59,
        5: 64
    }
    midi_min = str_midi_dict[string_number]- 10
    midi_max = str_midi_dict[string_number] + 30
    fmin = librosa.midi_to_hz(midi_min)
    fmax = librosa.midi_to_hz(midi_max)
    
    max_lag = librosa.time_to_samples(1./fmin, sr=sr)[0]
    min_lag = librosa.time_to_samples(1./fmax, sr=sr)[0]
    
    yf = librosa.util.frame(y, frame_length=max_lag, hop_length=256)
    A = librosa.autocorrelate(yf, axis=0)
    
    V = acf_to_voicing_prob(A)
    
    Corig = np.clip(A[min_lag:max_lag], 0, None) / A[0]
    C = np.vstack((Corig, 1-V))
    
    sigsq = np.mean(np.abs(C))
    P = np.exp(0.5 * C / sigsq)
    
    P = librosa.util.normalize(P.astype(np.float64), axis=0, norm=1)
 
    return P, min_lag

In [5]:
def compute_novelty_spec_flux(y, fs, hop_size=256):
    """ Compute novelty function using spectural flux.
    Parameters
    ----------
    y : ndarray
    (T,) mono time domian signal
    fs : int
    sample rate of y
    win_size : int
    hop_size : int
    
    Returns
    -------
    novelty : ndarray
    times : ndarray
    fs_novelty : float
    """
    S = librosa.feature.melspectrogram(y=y, sr=fs, n_mels=256, hop_length=hop_size)
    S_dB = librosa.power_to_db(S, ref=np.max)
    dS_dt = np.diff(S,axis=1)
    # HW Rectify dS_dt
    for x in np.nditer(dS_dt, op_flags=['readwrite']):
        x[...] = max(x,0)
    # 
    
    weight_vec = np.append( np.zeros(128), np.ones(128))
    weight_vec += 1
    
    weighted_sf = np.dot(np.diag(weight_vec), dS_dt)
    
    # output gathering
    novelty = np.mean(weighted_sf, axis=0)
    fs_novelty = float(fs) / hop_size
    times = np.array([float(i) / fs_novelty for i in range(len(novelty))])
    
    return novelty, times, fs_novelty

def medfilt(x, k):
    """Apply a length-k median filter to a 1D array x.
    Boundaries are extended by repeating endpoints.
    """
    assert k % 2 == 1, "Median filter length must be odd."
    assert x.ndim == 1, "Input must be one-dimensional."
    k2 = (k - 1) // 2
    y = np.zeros ((len (x), k), dtype=x.dtype)
    y[:,k2] = x
    for i in range (k2):
        j = k2 - i
        y[j:,i] = x[:-j]
        y[:j,i] = x[0]
        y[:-j,-(i+1)] = x[j:]
        y[-j:,-(i+1)] = x[-1]
    return np.median (y, axis=1)

def onsets_from_novelty(novelty, times, fs, w_c=3, medfilt_len=11, offset=0.001):
    # smoothing using butterworth and normalize
    nyq = fs / 2.0
    B, A = signal.butter(1, w_c / nyq, 'low')
    novelty_smoothed = signal.filtfilt(B, A, novelty)
    novelty_smoothed = novelty_smoothed / np.max(np.abs(novelty_smoothed))
    
    # adaptive thresholding
    thresh = medfilt(novelty_smoothed, medfilt_len) + offset
    
    # onset detection
    peak_idx = librosa.util.peak_pick(novelty_smoothed, 3, 3, 3, 5, 0, 10)
    
    pruned_peak_idx = []
    for p_idx in peak_idx:
        if novelty_smoothed[p_idx] > thresh[p_idx]: # it made it
            pruned_peak_idx.append(p_idx)
    return pruned_peak_idx, novelty_smoothed, thresh

In [6]:
def f0seg_to_note(f_seg, start_frame, sr, big_ann):
    """Given a f0 segmented by onset, output the note associated"""
    
    s_frames = []
    e_frames = []
    for idx in range(len(f_seg)):
        f_frame = f_seg[idx]
        if f_seg[idx] <= 0: # unvoiced
            if idx == 0:
                continue
            elif f_seg[idx - 1] <= 0:
                continue
            else: # just switched from voiced to unvoiced
                note_end_frame = start_frame + idx
                e_frames.append(note_end_frame)
        else: # voiced          
            if idx == 0:
                note_start_frame = start_frame + idx
                s_frames.append(note_start_frame)
            elif f_seg[idx - 1] <= 0: # just switched from unvoiced to voiced
                note_start_frame = start_frame + idx
                s_frames.append(note_start_frame)
            else:
                continue
    
    if len(e_frames) != len(s_frames):
        e_frames.append(start_frame + len(f_seg) - 1)
        
    s_times = librosa.frames_to_time(frames=s_frames, sr=sr, hop_length=256)
    e_times = librosa.frames_to_time(frames=e_frames, sr=sr, hop_length=256)    
    
          
    for idx in range(len(s_times)):
        if len(f_seg) == 0:
            continue
        pitch = np.mean(f_seg[s_frames[idx] - start_frame : e_frames[idx] - start_frame])
        print((s_times[idx], e_times[idx] - s_times[idx], pitch, None))
        big_ann.append(time=s_times[idx],
                   duration=e_times[idx] - s_times[idx],
                   value=pitch,
                   confidence=None)
    
    return big_ann


def detect_f0(y, sr, string_number, onset_frames=[]):
    pitch_prob, min_lag = constrained_pitch_prob(y, string_number)
    
    n_pitches = pitch_prob.shape[0]
    T = make_bump_transition(n_pitches, width=15, alpha=1e-3)
    
    onset_frames.insert(0, 0)
    
    f_detect = np.zeros((pitch_prob.shape[1], ))
    big_ann = jams.Annotation(namespace='note_hz')
    big_ann.duration = len(y) / float(sr)
    for frame in range(len(onset_frames)):
        print(frame, len(onset_frames))
        start_frame = onset_frames[frame]
        try:
            end_frame = onset_frames[frame+1]
        except IndexError:
            end_frame = None
        print('start:{} end:{}'.format(start_frame, end_frame))
        segment_pitch_prob = pitch_prob[:, start_frame:end_frame]
        if segment_pitch_prob.shape[1] == 0:
            continue
        seq, values, pointers = viterbi(segment_pitch_prob.T, T)
        f_detect_seg = float(sr) / (seq + min_lag)
        f_detect_seg[seq == n_pitches - 1] *= -1
        f_detect[start_frame:end_frame] = f_detect_seg
        
        # create note for each segment
        big_ann = f0seg_to_note(f_detect_seg, start_frame, sr, big_ann)
    return f_detect, big_ann

In [7]:
y, sr = librosa.load('/Users/tom/Desktop/eh_BN1-129-Eb_c_hex_cln_03.wav')

S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=256)
sf, times_on, fs_novelty_on = compute_novelty_spec_flux(y, sr)
onset_frames, novelty_smoothed, thresh = onsets_from_novelty(sf, times_on, fs_novelty_on)

In [8]:
f_detect, big_ann = detect_f0(y, sr, 3, onset_frames=onset_frames)

(0, 27)
start:0 end:62
(1, 27)
start:62 end:120
(0.71981859410430837, 0.61532879818594111, 314.83258038798829, None)
(2, 27)
start:120 end:181
(1.3931972789115645, 0.46439909297052173, 296.19057756386525, None)
(3, 27)
start:181 end:243
(2.10140589569161, 0.70820861678004521, 266.33909922928171, None)
(4, 27)
start:243 end:260
(2.8212244897959184, 0.18575963718820843, 264.08614641492801, None)
(5, 27)
start:260 end:282
(3.0185941043083901, 0.24380952380952348, 294.97064528643472, None)
(6, 27)
start:282 end:419
(3.2740136054421769, 0.30185941043083897, 262.99557203171662, None)
(7, 27)
start:419 end:442
(8, 27)
start:442 end:500
(5.1316099773242634, 0.56888888888888811, 298.30849685301735, None)
(9, 27)
start:500 end:580
(5.8049886621315192, 0.91718820861678019, 264.7838653161582, None)
(10, 27)
start:580 end:598
(6.7337868480725627, 0.19736961451247126, 296.8106016232951, None)
(11, 27)
start:598 end:1021
(6.9427664399092972, 0.19736961451247126, 268.23766254308953, None)
(12, 27)
sta

In [9]:
jam = jams.JAMS()
jam.annotations.append(big_ann)

In [11]:
big_ann

,time,duration,value,confidence
0,0.720,0.615,314.832580388,None
1,1.393,0.464,296.190577564,None
2,2.101,0.708,266.339099229,None
3,2.821,0.186,264.086146415,None
4,3.019,0.244,294.970645286,None
5,3.274,0.302,262.995572032,None
6,5.132,0.569,298.308496853,None
7,5.805,0.917,264.783865316,None
8,6.734,0.197,296.810601623,None
9,6.943,0.197,268.237662543,None


In [13]:
sr = 44100
data = jams.sonify.sonify(big_ann, sr)
Audio(data=data, rate=sr)


IndexError: index 18 is out of bounds for axis 0 with size 18

In [9]:
# plt.figure()
# librosa.display.specshow(librosa.power_to_db(S, ref=np.max), y_axis='mel', sr=sr, x_axis='time')
# times = librosa.frames_to_time(np.arange(len(f_detect)), sr=sr)
# mir_eval.display.pitch(times, f_detect, color='b', linewidth=3)
# plt.show()

In [37]:
times = librosa.frames_to_time(np.arange(len(f_detect)), sr=sr)
son = mir_eval.sonify.pitch_contour(times, f_detect, fs=sr, length=len(y))
Audio(data=data, rate=sr)

TypeError: piano_roll() got multiple values for keyword argument 'length'

In [ ]:
np.zeros??

In [40]:
test_jams_ann = jams.Annotation(namespace='note_hz')

In [41]:
test_jams_ann.duration = 22.0

In [42]:
test_jams_ann

<Annotation: annotation_metadata, data, sandbox, namespace, time, duration>

In [43]:
test_jams_ann.append?

In [44]:
test_jams_ann.append(time=0, duration=1, value='32')

In [47]:
test_jams_ann.data

,time,duration,value,confidence
0,0 days,00:00:01,00:00:00.000000,NaT
